# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [2]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 5.52MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
  Created wheel for Chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7095 sha256=690583b36747cf8764544170dd36fb841876db6541dc22356482c1cab32a2646
  Stored in directory: /root/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built Chroma


In [4]:
 !pip uninstall LLMChain

In [7]:
!pip install langchain

In [6]:
!pip install langchain-groq

In [31]:
from langchain.chains import LLMChain
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import pandas as pd
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [16]:
df = pd.read_csv("Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [9]:
df.duplicated().sum()

0

In [10]:
df.isna().sum()

,0
المخالفة,0
الغرامة,0


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [18]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [12]:
markdown_list = []

for index, row in df.iterrows():
    violation = row['المخالفة']
    fine = row['الغرامة']
    markdown = f"المخالفة: {violation} - الغرامة: {fine}"
    markdown_list.append(markdown)

    with open(f'{directory}/{index}.md', 'w', encoding='utf-8') as file:
        file.write(markdown)

In [13]:
markdown_list[5:10]

['المخالفة: وقوف المركبة في أماكن غير مخصصة للوقوف. - الغرامة: الغرامة المالية 100 - 150 ريال',
 'المخالفة: عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. - الغرامة: الغرامة المالية 100 - 150 ريال',
 'المخالفة: عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر. - الغرامة: الغرامة المالية 150 - 300 ريال']

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [21]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.create_documents(markdown_list)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [37]:
embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Nli-Matryoshka")
db = Chroma.from_documents(chunks, embeddings_model, persist_directory="./chroma_db")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [28]:

#list of the content from the most similar documents
def search_chroma_database(query_text, database, num_results=5):
    # Search the Chroma database for similar documents
    similar_docs = database.similarity_search(query_text, k=num_results)

    # Extract the content from the retrieved documents
    search_results = [document.page_content for document in similar_docs]

    return search_results


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [38]:
# Define the template for the prompt, where the {query} represents the user's question
PROMPT_TEMPLATE = """
السؤال: {query}
الغرامة: {context}
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "query"]
)


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [33]:
# from langchain.llms import Groq
from langchain_groq import ChatGroq

In [41]:
# Set the API key .
groq_api_key = "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"

#
# The model selected is "llama3-8b-8192"
llm = ChatGroq(
    temperature=0.7,  #
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192"
)


## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [35]:
# The 'verbose=1' parameter enables detailed logging,
#which is helpful for debugging and tracking the model's responses
rag_chain = LLMChain(llm=llm,
                     prompt=prompt_template,
                     verbose=1)

<ipython-input-35-fc4952293c32>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  rag_chain = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [36]:
def query_rag(query: str):
    # Perform a similarity search in the Chroma database with a score, retrieving the top 4 results.
    similarity_search_results = db.similarity_search_with_score(query, k=4)

    # Combine the content of the retrieved documents into a single context string,
    # separated by new lines for better readability in the prompt.
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    # Invoke the RAG chain, providing both the context and the query to the model for generating a response.
    rag_response = rag_chain.invoke({"context": context_text, "query": query})

    # Return the generated response from the RAG chain.
    return rag_response


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [39]:
# Test the RAG system by passing a user query asking about the fine for speeding.

user_query = "ما هي غرامةالسرعة؟"
response = query_rag(user_query)

# Print or return the generated response from the RAG model.
response


Prompt after formatting:

ما هي مخالفة السرعة؟: ما هي غرامةالسرعة؟
الغرامة: المخالفة: ملاحقة مركبات الطوارئ أثناء استعمال المنبهات الخاصة بها. - الغرامة: الغرامة المالية 500 - 900 ريال

المخالفة: استعمال المركبة لغير الغرض الذي رخصت من أجله. - الغرامة: الغرامة المالية 500 - 900 ريال

المخالفة: قيام سائق مركبة الطوارئ باستعمال المنبهات الخاصة بها من غير ضرورة. - الغرامة: الغرامة المالية 500 - 900 ريال

المخالفة: الانشغال بغير الطريق أثناء قيادة المركبة. - الغرامة: الغرامة المالية 300 - 500 ريال


> Finished chain.


{'context': 'المخالفة: ملاحقة مركبات الطوارئ أثناء استعمال المنبهات الخاصة بها. - الغرامة: الغرامة المالية 500 - 900 ريال\n\nالمخالفة: استعمال المركبة لغير الغرض الذي رخصت من أجله. - الغرامة: الغرامة المالية 500 - 900 ريال\n\nالمخالفة: قيام سائق مركبة الطوارئ باستعمال المنبهات الخاصة بها من غير ضرورة. - الغرامة: الغرامة المالية 500 - 900 ريال\n\nالمخالفة: الانشغال بغير الطريق أثناء قيادة المركبة. - الغرامة: الغرامة المالية 300 - 500 ريال',
 'query': 'ما هي غرامةالسرعة؟',
 'text': 'Here are the transgressions and their corresponding fines:\n\n1. **Mischief**: Using emergency vehicle sirens during personal use. **Fine**: SAR 500-900.\n2. **Mischief**: Using a vehicle for a purpose other than what it was licensed for. **Fine**: SAR 500-900.\n3. **Mischief**: Using emergency vehicle sirens unnecessarily. **Fine**: SAR 500-900.\n4. **Distraction**: Engaging in an activity unrelated to driving while operating a vehicle. **Fine**: SAR 300-500.\n\nNote: SAR stands for Saudi Riyal, which is the